In [17]:
%pip install vaderSentiment

     -------------------------------------- 126.0/126.0 kB 3.6 MB/s eta 0:00:00
Note: you may need to restart the kernel to use updated packages.



[notice] A new release of pip available: 22.3.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [21]:
from dash import Dash, html, dcc, callback, Output, Input, State
from vaderSentiment.vaderSentiment import SentimentIntensityAnalyzer

app = Dash()

analyzer = SentimentIntensityAnalyzer()

def analyse_sentiment(text):
    scores = analyzer.polarity_scores(text)
    return round(scores["compound"], 2)   # a number from -1 to +1

def sentiment_to_color(score):
    # clamp score just in case
    s = max(min(score, 1), -1)

    if s >= 0:
        intensity = int(s * 255)
        return f"rgb(0, {intensity}, 0)"
    else:
        intensity = int(-s * 255)
        return f"rgb({intensity}, 0, 0)"


app.layout = html.Div([
    html.H1("Sentiment Analyser"),

    dcc.Input(id="input-text", type="text", placeholder="Enter text..."),
    html.Button("Analyse Sentiment", id="analyse-btn", n_clicks=0),

    html.Div(id="sentiment-output", style={
        "marginTop": "20px",
        "padding": "15px",
        "borderRadius": "10px",
        "color": "white",
        "fontWeight": "bold"
    })
])

@callback(
    Output("sentiment-output", "children"),
    Output("sentiment-output", "style"),
    Input("analyse-btn", "n_clicks"),
    State("input-text", "value"),
    prevent_initial_call=True
)
def run_analysis(n_clicks, text):
    if not text:
        return "Please enter text.", {"backgroundColor": "white"}

    score = analyse_sentiment(text)
    color = sentiment_to_color(score)

    style = {
        "marginTop": "20px",
        "padding": "15px",
        "borderRadius": "10px",
        "color": "white",
        "fontWeight": "bold",
        "backgroundColor": color
    }

    return f"Sentiment score: {score}", style


if __name__ == "__main__":
    app.run()